In [2]:
import xgboost as xgb
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import lightgbm as lgb
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score 
from sklearn.metrics import make_scorer, mean_squared_error, log_loss

In [3]:
train = pd.read_csv(r"C:\Ziyuan Sui\MSBA\Fall22\ML1\project\data\ProjectManipulatedTrainingData.csv")

In [4]:
train = train.drop(columns=['id','device_id'])

In [5]:
label_encoder = LabelEncoder()

In [6]:
train['site_id']=label_encoder.fit_transform(train['site_id'])
train['site_domain']=label_encoder.fit_transform(train['site_domain'])
train['site_category']=label_encoder.fit_transform(train['site_category'])
train['app_id']=label_encoder.fit_transform(train['app_id'])
train['app_domain']=label_encoder.fit_transform(train['app_domain'])
train['app_category']=label_encoder.fit_transform(train['app_category'])
train['device_model']=label_encoder.fit_transform(train['device_model'])
train['device_type']=label_encoder.fit_transform(train['device_type'])
train['device_conn_type']=label_encoder.fit_transform(train['device_conn_type'])
train['C14']=label_encoder.fit_transform(train['C14'])
train['C15']=label_encoder.fit_transform(train['C15'])
train['C16']=label_encoder.fit_transform(train['C16'])
train['C17']=label_encoder.fit_transform(train['C17'])
train['C19']=label_encoder.fit_transform(train['C19'])
train['C20']=label_encoder.fit_transform(train['C20'])
train['C21']=label_encoder.fit_transform(train['C21'])

In [7]:
newtrain = train.iloc[:255929]
newval = train.iloc[255929:]

In [8]:
X_train = newtrain.drop(columns='click')
y_train = newtrain.loc[:,'click']
X_test = newval.drop(columns='click')
y_test = newval.loc[:,'click']

XGBoost

In [19]:
# tuning 'learning_rate'
param_test1 = { 
 'learning_rate':[0.05,0.1,0.15,0.2,0.25,0.3]
}
gsearch = GridSearchCV(
    estimator = XGBClassifier(objective= 'binary:logistic'), 
    param_grid = param_test1, 
    scoring= 'neg_log_loss',
    cv=5)
gsearch.fit(X_train,y_train)
gsearch.best_params_  

{'learning_rate': 0.25}

In [20]:
param_test2 = {
 'max_depth':[1,2,3,4,5,6,7,8,9,10]
}
gsearch = GridSearchCV(
    estimator = XGBClassifier(objective= 'binary:logistic', learning_rate=0.25), 
    param_grid = param_test2, 
    scoring= 'neg_log_loss',
    cv=5)
gsearch.fit(X_train,y_train)
gsearch.best_params_     

{'max_depth': 6}

In [21]:
# tuning 'min_child_weight'
param_test3 = {
 'min_child_weight':[1,2,3,4,5,6,7,8,9,10]
}
gsearch3 = GridSearchCV(
    estimator = XGBClassifier(objective= 'binary:logistic', learning_rate=0.25 , max_depth = 6), 
    param_grid = param_test3, 
    scoring= 'neg_log_loss',
    cv=5)
gsearch3.fit(X_train,y_train)
gsearch3.best_params_     

{'min_child_weight': 1}

In [22]:
dtrain = xgb.DMatrix(data=X_train,label=y_train)
dval = xgb.DMatrix(data=X_test,label=y_test)

param = { 'objective': 'binary:logistic', 'learning_rate':0.25, 'max_depth':6,'min_child_weight':1}
num_round = 1500
evallist = [(dval, 'eval'), (dtrain, 'train')]
bst = xgb.train(param, dtrain, num_round, evallist,early_stopping_rounds=80)

[0]	eval-logloss:0.58955	train-logloss:0.58992
[1]	eval-logloss:0.52828	train-logloss:0.52897


c:\Users\jacks\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:617: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[2]	eval-logloss:0.48964	train-logloss:0.49055
[3]	eval-logloss:0.46552	train-logloss:0.46660
[4]	eval-logloss:0.44980	train-logloss:0.45105
[5]	eval-logloss:0.43984	train-logloss:0.44117
[6]	eval-logloss:0.43281	train-logloss:0.43412
[7]	eval-logloss:0.42798	train-logloss:0.42938
[8]	eval-logloss:0.42478	train-logloss:0.42623
[9]	eval-logloss:0.42217	train-logloss:0.42357
[10]	eval-logloss:0.41977	train-logloss:0.42109
[11]	eval-logloss:0.41832	train-logloss:0.41957
[12]	eval-logloss:0.41745	train-logloss:0.41858
[13]	eval-logloss:0.41625	train-logloss:0.41726
[14]	eval-logloss:0.41560	train-logloss:0.41651
[15]	eval-logloss:0.41512	train-logloss:0.41589
[16]	eval-logloss:0.41464	train-logloss:0.41527
[17]	eval-logloss:0.41409	train-logloss:0.41454
[18]	eval-logloss:0.41363	train-logloss:0.41387
[19]	eval-logloss:0.41344	train-logloss:0.41356
[20]	eval-logloss:0.41308	train-logloss:0.41306
[21]	eval-logloss:0.41282	train-logloss:0.41271
[22]	eval-logloss:0.41263	train-logloss:0.41241


In [23]:
ypredval = bst.predict(dval, ntree_limit=bst.best_ntree_limit)

c:\Users\jacks\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


In [26]:
sklearn.metrics.log_loss(y_test, ypredval, normalize=True, sample_weight=None, labels=None)

0.4290323089484148

CatBoost

In [18]:

iterations=range(49,55)
learning_rate=[0.26,0.28,0.30,0.32]
depth=range(2,10)
parameters = dict(iterations = iterations, learning_rate = learning_rate,depth=depth)

gs_cat = GridSearchCV(estimator=CatBoostClassifier(verbose=False),
                      param_grid=parameters, cv=5, n_jobs=-1, scoring='neg_log_loss')

train_data = X_train
train_labels = y_train.values
gs_cat = gs_cat.fit(train_data,train_labels)

print(" Parameter Tuning")
print("cross-validate auc: %.4f" % gs_cat.best_score_)
print("Optimal Parameter: ", gs_cat.best_params_)
print("Optimal Estimator: ", gs_cat.best_estimator_)

 Parameter Tuning
cross-validate auc: -0.4132
Optimal Parameter:  {'depth': 8, 'iterations': 54, 'learning_rate': 0.28}
Optimal Estimator:  <catboost.core.CatBoostClassifier object at 0x00000274F82BA320>


In [19]:

iterations=range(15,22)
learning_rate=[0.1,0.15,0.2,0.25,0.28,0.30]
depth=range(3,10)
parameters = dict(iterations = iterations, learning_rate = learning_rate,depth=depth)

gs_cat2 = GridSearchCV(estimator=CatBoostClassifier(verbose=False),
                      param_grid=parameters, cv=5, n_jobs=-1, scoring='neg_log_loss')

train_data = X_train
train_labels = y_train.values
gs_cat2 = gs_cat2.fit(train_data,train_labels)

print(" Parameter Tuning")
print("cross-validate auc: %.4f" % gs_cat2.best_score_)
print("Optimal Parameter: ", gs_cat2.best_params_)
print("Optimal Estimator: ", gs_cat2.best_estimator_)

 Parameter Tuning
cross-validate auc: -0.4156
Optimal Parameter:  {'depth': 9, 'iterations': 21, 'learning_rate': 0.28}
Optimal Estimator:  <catboost.core.CatBoostClassifier object at 0x00000274F8297970>


In [23]:

iterations=range(55,65)
depth=range(3,12)
parameters = dict(iterations = iterations,depth=depth)

gs_cat3 = GridSearchCV(estimator=CatBoostClassifier(verbose=False, learning_rate = 0.28),
                      param_grid=parameters, cv=5, n_jobs=-1, scoring='neg_log_loss')

train_data = X_train
train_labels = y_train.values
gs_cat3 = gs_cat3.fit(train_data,train_labels)

print(" Parameter Tuning")
print("cross-validate auc: %.4f" % gs_cat3.best_score_)
print("Optimal Parameter: ", gs_cat3.best_params_)
print("Optimal Estimator: ", gs_cat3.best_estimator_)

 Parameter Tuning
cross-validate auc: -0.4130
Optimal Parameter:  {'depth': 8, 'iterations': 62}
Optimal Estimator:  <catboost.core.CatBoostClassifier object at 0x00000274EE952A10>


In [29]:
cb = CatBoostClassifier(learning_rate=0.28,depth=8,iterations=62,verbose=False)

In [33]:

logloss = log_loss(y_test,cb.fit(X_train,y_train).predict_proba(X_test))
logloss


0.4108673856237964

LightGBM

In [40]:
lgb = LGBMClassifier()
param_test = {
 'num_leaves':range(35,75,5),
 'min_data_in_leaf':range(45,85,5),
 'reg_alpha': [2,3,4,5,6]
}

In [41]:
gs_lgb = GridSearchCV(estimator=lgb,
                  param_grid=param_test,
                  scoring='neg_log_loss',
                  cv=5,
                  n_jobs=-1)

gs_lgb= gs_lgb.fit(X_train,y_train)
print("\n Parameter Tuning #4")
print("Non-nested CV Accuracy: ", gs_lgb.best_score_)
print("Optimal Parameter: ", gs_lgb.best_params_)
print("Optimal Estimator: ", gs_lgb.best_estimator_)


 Parameter Tuning #4
Non-nested CV Accuracy:  -0.41134767146682877
Optimal Parameter:  {'min_data_in_leaf': 50, 'num_leaves': 70, 'reg_alpha': 2}
Optimal Estimator:  LGBMClassifier(min_data_in_leaf=50, num_leaves=70, reg_alpha=2)


In [18]:
lgb2 = LGBMClassifier(num_leaves=70,min_data_in_leaf=50,reg_alpha=2)
param_test2 = {
'n_estimators' : range(100,200,5)
}

In [19]:
gs_lgb2 = GridSearchCV(estimator=lgb2,
                  param_grid=param_test2,
                  scoring='neg_log_loss',
                  cv=5,
                  n_jobs=-1)

gs_lgb2= gs_lgb2.fit(X_train,y_train)
print("\n Parameter Tuning #4")
print("Non-nested CV Accuracy: ", gs_lgb2.best_score_)
print("Optimal Parameter: ", gs_lgb2.best_params_)
print("Optimal Estimator: ", gs_lgb2.best_estimator_)


 Parameter Tuning #4
Non-nested CV Accuracy:  -0.4113226182501098
Optimal Parameter:  {'n_estimators': 110}
Optimal Estimator:  LGBMClassifier(min_data_in_leaf=50, n_estimators=110, num_leaves=70,
               reg_alpha=2)


In [20]:
lgb3 = LGBMClassifier(num_leaves=70,min_data_in_leaf=50,reg_alpha=2,n_estimators=110)
param_test3 = {
'bagging_fraction':[i/10.0 for i in range(1,10)],
 'feature_fraction':[i/10.0 for i in range(1,10)]
}

In [21]:
gs_lgb3 = GridSearchCV(estimator=lgb3,
                  param_grid=param_test3,
                  scoring='neg_log_loss',
                  cv=5,
                  n_jobs=-1)

gs_lgb3= gs_lgb3.fit(X_train,y_train)

print("Non-nested CV Accuracy: ", gs_lgb3.best_score_)
print("Optimal Parameter: ", gs_lgb3.best_params_)
print("Optimal Estimator: ", gs_lgb3.best_estimator_)

Non-nested CV Accuracy:  -0.41106358904623097
Optimal Parameter:  {'bagging_fraction': 0.1, 'feature_fraction': 0.5}
Optimal Estimator:  LGBMClassifier(bagging_fraction=0.1, feature_fraction=0.5, min_data_in_leaf=50,
               n_estimators=110, num_leaves=70, reg_alpha=2)


In [17]:
lgb = LGBMClassifier(num_leaves=70,min_data_in_leaf=50,reg_alpha=2,n_estimators=110,bagging_fraction=0.1,feature_fraction=0.5)

In [9]:
logloss_lgb = log_loss(y_test,lgb.fit(X_train,y_train).predict_proba(X_test))
logloss_lgb

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=1.0 will be ignored. Current value: bagging_fraction=0.1


0.40849170783978106

In [21]:
test = pd.read_csv(r"C:\Ziyuan Sui\MSBA\Fall22\ML1\project\data\ProjectManipulatedTestDataFINAL.csv")

In [22]:
id_col = test.id

In [23]:
test = test.drop(columns=['id','device_id','device_ip','Unnamed: 0'])

In [25]:
label_encoder2 = LabelEncoder()

In [26]:
test['site_id']=label_encoder2.fit_transform(test['site_id'])
test['site_domain']=label_encoder2.fit_transform(test['site_domain'])
test['site_category']=label_encoder2.fit_transform(test['site_category'])
test['app_id']=label_encoder2.fit_transform(test['app_id'])
test['app_domain']=label_encoder2.fit_transform(test['app_domain'])
test['app_category']=label_encoder2.fit_transform(test['app_category'])
test['device_model']=label_encoder2.fit_transform(test['device_model'])
test['device_type']=label_encoder2.fit_transform(test['device_type'])
test['device_conn_type']=label_encoder2.fit_transform(test['device_conn_type'])
test['C14']=label_encoder2.fit_transform(test['C14'])
test['C15']=label_encoder2.fit_transform(test['C15'])
test['C16']=label_encoder2.fit_transform(test['C16'])
test['C17']=label_encoder2.fit_transform(test['C17'])
test['C19']=label_encoder2.fit_transform(test['C19'])
test['C20']=label_encoder2.fit_transform(test['C20'])
test['C21']=label_encoder2.fit_transform(test['C21'])

In [19]:
X_train.columns

Index(['C1', 'day', 'hour', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_model',
       'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18',
       'C19', 'C20', 'C21', 'weekday'],
      dtype='object')

In [24]:
test.columns

Index(['C1', 'day', 'hour', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_model',
       'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18',
       'C19', 'C20', 'C21', 'weekday'],
      dtype='object')

In [27]:
y_pred_real = lgb.fit(X_train,y_train).predict_proba(test)

In [37]:
y_pred_real

array([[0.85524023, 0.14475977],
       [0.89973901, 0.10026099],
       [0.89636786, 0.10363214],
       ...,
       [0.94696446, 0.05303554],
       [0.95387363, 0.04612637],
       [0.94009743, 0.05990257]])

In [34]:
final = pd.DataFrame(id_col,columns=['id'])

In [38]:
final['P(click)'] = y_pred_real[:,1]

In [40]:
final.shape

(13015341, 2)

In [41]:
final.to_csv(r"C:\Ziyuan Sui\MSBA\Fall22\ML1\project\data\ProjectSubmission-Team5.csv",index=False)